In [1]:
import pandas as pd

# CSV 파일 로드
file_path = 'C:/Users/USER/Desktop/products.csv'  # 파일 경로를 지정하세요
data = pd.read_csv(file_path)

# 추출할 품목 리스트
target_products = ['명태', '고등어', '오징어', '양파', '오이', '배추', '돼지고기', '닭고기']

# 품목 필터링 함수 (Title에서 대상 품목 이름이 포함된 행만 추출)
def filter_products(data, targets):
    # 대상 품목이 포함된 Title만 필터링 (NaN 또는 비문자열 데이터 처리)
    filtered_data = data[data['Title'].apply(lambda x: isinstance(x, str) and any(product in x for product in targets))]
    return filtered_data

# 필터링 실행
filtered_data = filter_products(data, target_products)

# 결과 출력
print(filtered_data)

# 필요한 경우 추출된 데이터 저장
output_file_path = 'filtered_products.csv'
filtered_data.to_csv(output_file_path, index=False)
print(f"추출된 데이터를 {output_file_path} 파일로 저장했습니다.")


                   Title  Price       Unit Price  \
84            (A)양배추(벌크)   4980     (1개당 4,980원)   
88            깨끗한 양배추(통)   4980     (1개당 4,980원)   
90           양파(1.5kg/망)   3980     (100g당 265원)   
91          백다다기오이(5입/봉)   5980     (1봉당 5,980원)   
97         못난이 양파(3kg/망)   6980     (100g당 233원)   
101         양배추(국산,1/4통)   1980     (1개당 1,980원)   
102             쌈용맛배추(팩)   3980     (1팩당 3,980원)   
127            손질배추(특/봉)   4480     (100g당 249원)   
133           취청오이(2입/봉)   4980     (1봉당 4,980원)   
138          GAP오이맛고추(팩)   3480     (1팩당 3,480원)   
191                깐양파/봉   3480     (100g당 535원)   
193        GAP백오이(3입/국산)   4980     (1봉당 4,980원)   
195          자색양파(1kg/망)   4980     (100g당 498원)   
199           적양배추(1/2통)   2480     (1개당 2,480원)   
505         국산손질오징어2미(팩)   9900   (100g당 2,750원)   
509   동그랗게 절단한 오징어링 500g   8980   (100g당 1,796원)   
510            노르웨이고등어필렛  12980   (100g당 1,298원)   
520       영일만건오징어3미(원양산)  10980   (100g당 9,982원)   
522     x 페스

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# CSV 파일 로드
file_path = 'C:/Users/USER/Desktop/products.csv'
data = pd.read_csv(file_path)

# 추출할 품목 리스트
target_products = ['명태', '고등어', '오징어', '양파', '오이', '배추', '돼지고기', '닭고기']

# TF-IDF 계산 (ngram_range 추가)
vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # 1~2개의 단어 조합 고려
tfidf_matrix = vectorizer.fit_transform(data['Title'].dropna())  # 결측값 제거

# 대상 품목 리스트에 대한 TF-IDF 계산
target_matrix = vectorizer.transform(target_products)

# 유사도 계산
similarity_scores = cosine_similarity(tfidf_matrix, target_matrix)

# 가장 유사한 품목 추출
data['max_similarity'] = similarity_scores.max(axis=1)
filtered_data = data[data['max_similarity'] > 0.005]  # 유사도 기준값(0.1) 설정

# 결과 출력
print(filtered_data[['Title', 'max_similarity']])


            Title  max_similarity
90    양파(1.5kg/망)        0.588899
97  못난이 양파(3kg/망)        0.458825


In [9]:
import pandas as pd

# CSV 파일 로드
file_path = 'C:/Users/USER/Desktop/products.csv'
data = pd.read_csv(file_path)

# 추출할 품목 리스트 (포함되어야 하는 품목 키워드)
target_products = ['명태', '고등어', '오징어', '양파', '오이', '배추', '돼지고기', '닭고기']

# 제외 조건 리스트 (제외되어야 하는 키워드)
exclude_keywords = ['맛', '가공', '조미', '채', '깡', '튀김']

# 필터링 함수 정의
def filter_products(data, targets, excludes):
    def is_valid_title(title):
        if not isinstance(title, str):  # 제목이 문자열이 아닐 경우 제외
            return False
        # 제외 키워드가 포함된 경우 False
        if any(exclude in title for exclude in excludes):
            return False
        # 대상 품목 키워드가 포함된 경우 True
        return any(target in title for target in targets)

    # 제목 기준 필터링 적용
    filtered_data = data[data['Title'].apply(is_valid_title)]
    return filtered_data

# 필터링 실행
filtered_data = filter_products(data, target_products, exclude_keywords)

# 결과 출력
print(filtered_data)




                   Title  Price       Unit Price  \
84            (A)양배추(벌크)   4980     (1개당 4,980원)   
88            깨끗한 양배추(통)   4980     (1개당 4,980원)   
90           양파(1.5kg/망)   3980     (100g당 265원)   
91          백다다기오이(5입/봉)   5980     (1봉당 5,980원)   
97         못난이 양파(3kg/망)   6980     (100g당 233원)   
101         양배추(국산,1/4통)   1980     (1개당 1,980원)   
127            손질배추(특/봉)   4480     (100g당 249원)   
133           취청오이(2입/봉)   4980     (1봉당 4,980원)   
191                깐양파/봉   3480     (100g당 535원)   
193        GAP백오이(3입/국산)   4980     (1봉당 4,980원)   
195          자색양파(1kg/망)   4980     (100g당 498원)   
199           적양배추(1/2통)   2480     (1개당 2,480원)   
505         국산손질오징어2미(팩)   9900   (100g당 2,750원)   
509   동그랗게 절단한 오징어링 500g   8980   (100g당 1,796원)   
510            노르웨이고등어필렛  12980   (100g당 1,298원)   
520       영일만건오징어3미(원양산)  10980   (100g당 9,982원)   
522     x 페스츄리BBQ오징어200g  11980   (100g당 5,990원)   
528        (균)꾸이오징어다리50g   3980   (100g당 7,960원)   
532         